In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import importlib
import log_parser
import scipy.stats as st
import importlib

from basic_tools import *

debug=False

# Loading h2 results

In [2]:
h2_neale_result=pd.read_csv(h2_path,sep='\t')
h2_neale_result=h2_neale_result.set_index('phenotype')
h2_neale_sorted=h2_neale_result.sort_values('h2_observed',ascending=False)
h2_neale_sorted.columns = h2_neale_sorted.columns.map(lambda x: 'neale.'+str(x))

In [13]:
h2_neale_v2_result=pd.read_csv(h2_v2_path,sep='\t')
h2_neale_v2_result=h2_neale_v2_result.set_index('phenotype')
h2_neale_v2_sorted=h2_neale_v2_result.sort_values('h2_observed',ascending=False)
h2_neale_v2_sorted.columns = h2_neale_v2_sorted.columns.map(lambda x: 'neale2.'+str(x))

## from my ldsc result of univariate

In [4]:
h2_myuni_sorted=read_ldsc('uni',False)
h2_myuni_sorted.head()

,uni.filename,uni.Total Observed scale h2,uni.Total Observed scale h2_se,uni.Total Liability scale h2,uni.Total Liability scale h2_se,uni.Lambda GC,uni.Mean Chi^2,uni.Intercept,uni.Intercept_se,uni.Ratio
phenotype,,,,,,,,,,
50_irnt,data/out_final/uni.50_irnt.ldsc.log,0.3772,0.0197,0.3772,0.0197,2.5074,4.7497,1.6108,0.0373,0.1629 (0.0099)
20153_irnt,data/out_final/uni.20153_irnt.ldsc.log,0.3293,0.0209,0.3293,0.0209,1.5696,2.1139,1.2127,0.0188,0.1909 (0.0169)
5097_irnt,data/out_final/uni.5097_irnt.ldsc.log,0.2878,0.0233,0.2878,0.0233,1.3169,1.5435,1.1186,0.0134,0.2183 (0.0246)
5098_irnt,data/out_final/uni.5098_irnt.ldsc.log,0.2825,0.0223,0.2825,0.0223,1.3203,1.5455,1.1236,0.0135,0.2266 (0.0247)
5133_irnt,data/out_final/uni.5133_irnt.ldsc.log,0.2771,0.0221,0.2771,0.0221,1.3068,1.5356,1.1237,0.0135,0.231 (0.0252)


## from my ldsc result of various BP scales

In [5]:
suffix_bp_list_power="bp300 bp128 bp64 bp32 bp16 bp8".split()
suffix_bp_list_full="bp300 bp128 bp64 bp32 bp16 bp8".split()

h2_bp_sorted_list=[]
for suffix in suffix_bp_list_full:
    h2_bp_sorted_list.append(read_ldsc(suffix,verbose=True))
    print(suffix,"was loaded")

bp300 was loaded
bp128 was loaded
bp64 was loaded
bp32 was loaded
bp16 was loaded
bp8 was loaded


## merging data.

In [6]:
h2_total=h2_bp_sorted_list[0].copy()
for h2_bp_sorted in h2_bp_sorted_list[1:]:
    h2_total=h2_total.join(h2_bp_sorted)

h2_total=h2_total.join(h2_myuni_sorted)

In [7]:
h2_total['uni.Total h2 z']=h2_total['uni.Total Observed scale h2']/h2_total['uni.Total Observed scale h2_se']
h2_total['uni.Total h2 p']=st.norm.sf(h2_total['uni.Total Observed scale h2']/h2_total['uni.Total Observed scale h2_se'])

In [11]:
h2_total.loc[phenotypes_par_filtered.index].to_csv(h2_total_par_filtered_path)

In [12]:
h2_total=h2_total.join(h2_neale_sorted)
h2_total=h2_total.join(h2_neale_v2_sorted)

NameError: name 'h2_neale_v2_sorted' is not defined

## Printing phenotypes that the three sources have in common

In [18]:
#h2_total['neale.h2_observed'], h2_total['atlas.h2'])al['my.Total Observed scale h2'])
conditions=[]
conditions.append(np.isnan(h2_total['neale.h2_observed']))
conditions.append(np.isnan(h2_total['neale2.h2_observed']))
conditions.append(np.isnan(h2_total['uni.Total Observed scale h2']))

for suffix in suffix_bp_list_full:
    conditions.append(np.isnan(h2_total['{}.Total Observed scale h2'.format(suffix)]))    
    
common=np.logical_not(np.logical_or.reduce(conditions))
print(np.sum(common))

186


In [19]:
h2_total=h2_total.loc[phenotypes_par_filtered.index]

# Verification

## Parsing result of univariate LDSC

In [20]:
pheno_code='23115_irnt'
regression_result_uni=parse_uni_regression_result(h2_total,pheno_code)
regression_result_uni

finished loading uni


{'h2': 0.1718,
 'h2_se': 0.0058,
 'intercept': 1.2555,
 'category': 'uni',
 'M': 6856128}

## Parsing result of partitioned LDSC

In [21]:
suffix_bp_list_power="bp300 bp128 bp64 bp32 bp16 bp8".split()

In [22]:
regression_result_bp_list_dict=dict([make_regression_result_list(h2_total,phenotype,suffix_bp_list_power) for phenotype in phenotypes_par_filtered.index])

In [23]:
for idx,suffix in enumerate(suffix_bp_list_power):
    print("{}: {}".format(suffix,regression_result_bp_list_dict['23115_irnt'][idx]['h2'].sum()))

bp300: 0.1881
bp128: 0.1895
bp64: 0.19039999999999999
bp32: 0.18999999999999997
bp16: 0.18877615
bp8: 0.18934133916999998


In [24]:
len(regression_result_bp_list_dict.keys())

497

In [26]:
h2_total_filtered.shape,h2_total.shape

NameError: name 'h2_total_filtered' is not defined